Summary of cleaning actions:
- Add names to the cities
- Drop columns that contain no data
- Convert timestamps to datetimes and set a datetime index
- In columns with partial data, assume Nans are zero values.
- convert temperature values from Kelvin to Celcius

In [1]:
import pandas as pd

In [81]:
#import data
path = './data/weather/spain-weather-2013-2019.csv'
data = pd.read_csv(path)

In [82]:
#first look at the data
data.head(3)

,dt,dt_iso,city_id,city_name,lat,lon,temp,temp_min,temp_max,pressure,...,rain_today,snow_1h,snow_3h,snow_24h,snow_today,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1380585600,2013-10-01 00:00:00 +0000 UTC,2509954,NaN,NaN,NaN,299.150,299.150,299.150,1008,...,NaN,NaN,NaN,NaN,NaN,20,801,Clouds,few clouds,02n
1,1380589200,2013-10-01 01:00:00 +0000 UTC,2509954,NaN,NaN,NaN,298.150,298.150,298.150,1009,...,NaN,NaN,NaN,NaN,NaN,20,801,Clouds,few clouds,02n
2,1380592800,2013-10-01 02:00:00 +0000 UTC,2509954,NaN,NaN,NaN,296.161,296.161,296.161,1009,...,NaN,NaN,0.0,NaN,NaN,10,800,Clear,sky is Clear,02


In [83]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263000 entries, 0 to 262999
Data columns (total 28 columns):
dt                     263000 non-null int64
dt_iso                 263000 non-null object
city_id                263000 non-null int64
city_name              0 non-null float64
lat                    0 non-null float64
lon                    0 non-null float64
temp                   263000 non-null float64
temp_min               263000 non-null float64
temp_max               263000 non-null float64
pressure               263000 non-null int64
sea_level              0 non-null float64
grnd_level             0 non-null float64
humidity               263000 non-null int64
wind_speed             263000 non-null int64
wind_deg               263000 non-null int64
rain_1h                27406 non-null float64
rain_3h                20017 non-null float64
rain_24h               0 non-null float64
rain_today             0 non-null float64
snow_1h                2 non-null float64
snow

In [85]:
def get_clean_weather(data):
    
    #add city names
    city_codes = {3128760 : ' Barcelona', 
                  3117735 : 'Madrid', 
                  3128026 : 'Bilbao', 
                  2509954 : 'Valencia', 
                  6361046 : 'Seville'}
    data['city_name'] = data['city_id'].replace(city_codes)

    #drop all columns with only NaN values
    data = data.drop(['lat', 
                      'lon', 
                      'sea_level', 
                      'grnd_level', 
                      'rain_24h', 
                      'snow_today',
                      'rain_today', 
                      'snow_1h', 
                      'snow_24h'], axis=1)


    #convert timestamp to datetime object
    times = pd.to_datetime(data['dt'], unit='s', origin='unix')

    #convert the times to local time zone
    data['dt'] = times.dt.tz_localize('UTC').dt.tz_convert('Europe/Madrid').dt.strftime('%Y-%m-%d %H:%M:%S')


    nul_cols = ['rain_1h', 'rain_3h', 'snow_3h']
    data[nul_cols] = data[nul_cols].fillna(0)
    
    return data

In [89]:
data = get_clean_weather(data)
data.head(3)

,dt,dt_iso,city_id,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,2013-10-01 02:00:00,2013-10-01 00:00:00 +0000 UTC,2509954,Valencia,299.150,299.150,299.150,1008,61,5,290,0.0,0.0,0.0,20,801,Clouds,few clouds,02n
1,2013-10-01 03:00:00,2013-10-01 01:00:00 +0000 UTC,2509954,Valencia,298.150,298.150,298.150,1009,65,4,250,0.0,0.0,0.0,20,801,Clouds,few clouds,02n
2,2013-10-01 04:00:00,2013-10-01 02:00:00 +0000 UTC,2509954,Valencia,296.161,296.161,296.161,1009,71,4,269,0.0,0.0,0.0,10,800,Clear,sky is Clear,02


In [87]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263000 entries, 0 to 262999
Data columns (total 19 columns):
dt                     263000 non-null object
dt_iso                 263000 non-null object
city_id                263000 non-null int64
city_name              263000 non-null object
temp                   263000 non-null float64
temp_min               263000 non-null float64
temp_max               263000 non-null float64
pressure               263000 non-null int64
humidity               263000 non-null int64
wind_speed             263000 non-null int64
wind_deg               263000 non-null int64
rain_1h                263000 non-null float64
rain_3h                263000 non-null float64
snow_3h                263000 non-null float64
clouds_all             263000 non-null int64
weather_id             263000 non-null int64
weather_main           263000 non-null object
weather_description    263000 non-null object
weather_icon           263000 non-null object
dtypes: float64(

### Converting temperatures

The min and max temperatures look to be the same as the hourly temperature indicated. We will check to see if this is always the case. If this is the case these columns will be removed.